# Proyecto Final
Desarrollo de un Modelo de Predicción de Tasa de la Cancelación de Clientes para la empresa Interconnect

# Introducción
 
Interconnect, operador en el sector de telecomunicaciones, busca optimizar sus estrategias de retención de clientes mediante la implementación de un modelo de analítica predictiva. En particular, el objetivo es anticipar la cancelación de contratos por parte de los usuarios, lo cual permitirá a la compañía actuar proactivamente a través de ofertas personalizadas, promociones o ajustes en sus planes comerciales.

Para ello, se ha planteado el desarrollo de un modelo de clasificación binaria que, a partir del análisis del comportamiento histórico de los clientes, identifique aquellos con mayor probabilidad de abandonar la compañía.
 
El objetivo principal de este análisis es construir un modelo de predicción que maximice la capacidad de detección de cancelaciones con base en información disponible. La selección del modelo óptimo se basará en su desempeño, siendo la métrica principal el AUC-ROC, debido a su capacidad para evaluar la discriminación entre clases. Esta métrica será complementada con otras como la exactitud (accuracy) para brindar una visión integral del rendimiento del modelo.

# Inicialización

In [1]:
# Librerias

import warnings
# Suppress all FutureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_regression
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.utils import resample

from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier


# Carga de archivos
 
Descripción de los archivos proporcionados con los cuales vamos a trabajar:
 
 - `contract.csv`: información del contrato.
 - `personal.csv`: datos personales del cliente.
 - `internet.csv`: información sobre los servicios de Internet.
 - `phone.csv`: información sobre los servicios telefónicos.

In [2]:
df_contract = pd.read_csv('/datasets/final_provider/contract.csv')
df_personal = pd.read_csv('/datasets/final_provider/personal.csv')
df_internet = pd.read_csv('/datasets/final_provider/internet.csv')
df_phone = pd.read_csv('/datasets/final_provider/phone.csv')


# Análisis Exploratorio de Datos de df_contract
 
Con el objetivo de garantizar la calidad y consistencia de la información disponible, se lleva a cabo una fase inicial de análisis exploratorio de datos. Esta etapa permite validar la estructura del conjunto de datos y detectar posibles inconsistencias que puedan afectar el desarrollo del modelo predictivo.
 
Para ello, se utilizarán funciones estándar de la biblioteca `pandas`, tales como:
 
 - `sample()` – para obtener una vista preliminar de observaciones aleatorias del dataset y comprender su estructura general.  
 - `info()` – para revisar los tipos de datos, la integridad de las columnas y la presencia de valores nulos.  
 - `describe()` – para generar estadísticas descriptivas que permitan evaluar la distribución y comportamiento de las variables numéricas.
 
Este análisis preliminar es fundamental para identificar si se requieren acciones adicionales de limpieza, transformación o imputación de datos antes de avanzar hacia las etapas de preprocesamiento y modelado. Una vez depurados, los distintos conjuntos de datos serán integrados en un único dataset maestro que consolide la información relevante por cliente, habilitando así una base sólida para el entrenamiento del modelo de predicción.

In [3]:
df_contract.head()

,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,7590-VHVEG,2020-01-01,No,Month-to-month,Yes,Electronic check,29.85,29.85
1,5575-GNVDE,2017-04-01,No,One year,No,Mailed check,56.95,1889.5
2,3668-QPYBK,2019-10-01,2019-12-01 00:00:00,Month-to-month,Yes,Mailed check,53.85,108.15
3,7795-CFOCW,2016-05-01,No,One year,No,Bank transfer (automatic),42.30,1840.75
4,9237-HQITU,2019-09-01,2019-11-01 00:00:00,Month-to-month,Yes,Electronic check,70.70,151.65


In [4]:
df_contract.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   BeginDate         7043 non-null   object 
 2   EndDate           7043 non-null   object 
 3   Type              7043 non-null   object 
 4   PaperlessBilling  7043 non-null   object 
 5   PaymentMethod     7043 non-null   object 
 6   MonthlyCharges    7043 non-null   float64
 7   TotalCharges      7043 non-null   object 
dtypes: float64(1), object(7)
memory usage: 440.3+ KB


<div class="alert alert-block alert-warning">
<b>Celda [4]</b> <a class="tocSkip"></a><br>
La columna 'TotalCharges' se cargó inicialmente como 'object' en lugar de 'float'. Aunque corregiste esto más adelante, es fundamental revisar y ajustar los tipos de datos justo después de cargar el dataset para evitar problemas en cálculos posteriores.
</div>


In [5]:
df_contract.describe()

,MonthlyCharges
count,7043.000000
mean,64.761692
std,30.090047
min,18.250000
25%,35.500000
50%,70.350000
75%,89.850000
max,118.750000


# Limpieza de df_contract
Con base al EDA y a buenas practicas, se realizarán los siguientes pasos:
 
 - Cambiar los nombres de las columnas a minúsculas y con formato snake_case para estandarizar.
 - Corregir los tipos de datos en las columnas:
   - Columnas de fechas (`begindate` y `enddate`) al formato datetime.
   - Columna `totalcharges` a tipo numérico (float).
 - Verificar la presencia de registros duplicados.


In [6]:
#función para la conversión de nombres de columnas a minisculas con formato snake_case

def column_snake_case(df):
    df.columns = (
        df.columns
        .str.replace(r'([a-z])([A-Z])', r'\1_\2', regex=True)
        .str.lower()
    )
    return df

column_snake_case(df_contract)

# corroborar el nombre de las funciones
df_contract.columns

Index(['customer_id', 'begin_date', 'end_date', 'type', 'paperless_billing',
       'payment_method', 'monthly_charges', 'total_charges'],
      dtype='object')

In [7]:
# conversión de las columnas 'begin_date y end_date' al formato datetime
df_contract['begin_date'] = pd.to_datetime(df_contract['begin_date'], errors='coerce')
df_contract['end_date'] = pd.to_datetime(df_contract['end_date'], errors='coerce')

df_contract[['begin_date', 'end_date']].dtypes

begin_date    datetime64[ns]
end_date      datetime64[ns]
dtype: object

In [8]:
# conversión de 'total_charges' a tipo númerico.
df_contract['total_charges'] = pd.to_numeric(df_contract['total_charges'], errors='coerce')

# corroborar el cambio
df_contract[['total_charges']].dtypes

total_charges    float64
dtype: object

In [9]:
# validación de datos nulos (NAN)
print(df_contract.isnull().sum())

customer_id             0
begin_date              0
end_date             5174
type                    0
paperless_billing       0
payment_method          0
monthly_charges         0
total_charges          11
dtype: int64


Observamos que existen 5174 valores nulos, sin embargo, son clientes activos. Más adelante esta columna se convertira en una columna binaria para identificar activos y bajas.

In [10]:
# vamos a analizar los valores nulos en 'total_charges' para decidir cómo imputarlos.
df_contract[df_contract['total_charges'].isnull()]

,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges
488,4472-LVYGI,2020-02-01,NaT,Two year,Yes,Bank transfer (automatic),52.55,NaN
753,3115-CZMZD,2020-02-01,NaT,Two year,No,Mailed check,20.25,NaN
936,5709-LVOEQ,2020-02-01,NaT,Two year,No,Mailed check,80.85,NaN
1082,4367-NUYAO,2020-02-01,NaT,Two year,No,Mailed check,25.75,NaN
1340,1371-DWPAZ,2020-02-01,NaT,Two year,No,Credit card (automatic),56.05,NaN
3331,7644-OMVMY,2020-02-01,NaT,Two year,No,Mailed check,19.85,NaN
3826,3213-VVOLG,2020-02-01,NaT,Two year,No,Mailed check,25.35,NaN
4380,2520-SGTTA,2020-02-01,NaT,Two year,No,Mailed check,20.00,NaN
5218,2923-ARZLG,2020-02-01,NaT,One year,Yes,Mailed check,19.70,NaN
6670,4075-WKNIU,2020-02-01,NaT,Two year,No,Mailed check,73.35,NaN


Podemos observar que los datos nulos en la columna `total_charges` corresponden a clientes que recién comenzaron su contrato con la compañía, por lo que aún no tienen un importe facturado. Por este motivo, se imputarán con 0.

In [11]:
# imputar valores NAN con 0 en columna 'total_charges'
df_contract['total_charges'] = df_contract['total_charges'].fillna(0)

# corroborar que ya no haya NAN
df_contract['total_charges'].isnull().sum()

0

In [12]:
# validación de duplicados
df_contract.duplicated().sum()

0

**Con todo esto, dejamos df_contract limpio y listo para integrarse posteriormente con los demás datasets.**

# Análisis Exploratorio de Datos de df_personal

In [13]:
df_personal.sample(5)

,customerID,gender,SeniorCitizen,Partner,Dependents
790,7131-ZQZNK,Female,0,Yes,Yes
3942,6959-UWKHF,Male,0,No,No
5944,5995-OIGLP,Male,0,No,No
1144,0841-NULXI,Male,0,No,No
1027,6732-FZUGP,Female,0,No,No


In [14]:
df_personal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   customerID     7043 non-null   object
 1   gender         7043 non-null   object
 2   SeniorCitizen  7043 non-null   int64 
 3   Partner        7043 non-null   object
 4   Dependents     7043 non-null   object
dtypes: int64(1), object(4)
memory usage: 275.2+ KB


In [15]:
df_personal.describe()

,SeniorCitizen
count,7043.000000
mean,0.162147
std,0.368612
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


# Limpieza de df_personal
En este dataset no será necesario cambiar los tipos de datos, ya que todos se encuentran correctamente definidos.
 
 Se realizarán las siguientes acciones:
 - Estandarizar los nombres de las columnas a formato `snake_case`.
 - Verificar si existen valores ausentes o nulos en alguna columna y, en caso de encontrarlos, analizar la mejor forma de imputación.
 - Revisar si existen registros duplicados.

In [16]:
#Conversión de nombres de columnas a minisculas con formato snake case.
column_snake_case(df_personal)
print(df_personal.columns)

Index(['customer_id', 'gender', 'senior_citizen', 'partner', 'dependents'], dtype='object')


In [17]:
#Validación de datos nulos (NA)
print(df_personal.isnull().sum())

customer_id       0
gender            0
senior_citizen    0
partner           0
dependents        0
dtype: int64


In [18]:
#Validación de duplicados
df_contract.duplicated().sum()

0

**Este dataset está listo para integrarse a los demas datasets.**

# Análisis Exploratorio de Datos de df_internet

In [19]:
df_internet.sample(5)

,customerID,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
2567,9626-VFRGG,Fiber optic,No,Yes,No,No,Yes,No
1285,6035-RIIOM,Fiber optic,No,Yes,No,No,Yes,Yes
845,0376-YMCJC,Fiber optic,No,No,No,Yes,Yes,No
3400,1518-OMDIK,DSL,No,No,No,No,No,No
2783,8417-GSODA,Fiber optic,No,No,No,No,Yes,Yes


In [20]:
df_internet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5517 entries, 0 to 5516
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   customerID        5517 non-null   object
 1   InternetService   5517 non-null   object
 2   OnlineSecurity    5517 non-null   object
 3   OnlineBackup      5517 non-null   object
 4   DeviceProtection  5517 non-null   object
 5   TechSupport       5517 non-null   object
 6   StreamingTV       5517 non-null   object
 7   StreamingMovies   5517 non-null   object
dtypes: object(8)
memory usage: 344.9+ KB


In [21]:
df_internet.describe()

,customerID,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
count,5517,5517,5517,5517,5517,5517,5517,5517
unique,5517,2,2,2,2,2,2,2
top,4879-GZLFH,Fiber optic,No,No,No,No,No,No
freq,1,3096,3498,3088,3095,3473,2810,2785


# Limpieza de df_internet
 
 - Estandarizar los nombres de las columnas a formato `snake_case`.
 - Verificar si existen valores nulos o ausentes y definir la estrategia de imputación si es necesario.
 - Revisar si hay registros duplicados.

In [22]:
#Conversión de nombres de columnas a minisculas.
column_snake_case(df_internet)
print(df_internet.columns)

Index(['customer_id', 'internet_service', 'online_security', 'online_backup',
       'device_protection', 'tech_support', 'streaming_tv',
       'streaming_movies'],
      dtype='object')


In [23]:
#Validación de datos nulos (NA)
print(df_internet.isnull().sum())

customer_id          0
internet_service     0
online_security      0
online_backup        0
device_protection    0
tech_support         0
streaming_tv         0
streaming_movies     0
dtype: int64


In [24]:
#Validación de duplicados
df_internet.duplicated().sum()

0

**Este dataset está listo para integrarse a los otros datasets. Sin embargo, contiene menos registros (5,517) que los otros datasets revisados anteriormente. Esto se debe a que no todos los clientes cuentan con servicios de Internet. Será necesario tener en cuenta este detalle al momento de realizar la unión final de los datos.**

# Análisis Exploratorio de Datos de df_phone

In [25]:
df_phone.sample(5)

,customerID,MultipleLines
1382,9207-ZPANB,Yes
3617,2860-RANUS,Yes
4621,7269-JISCY,No
2672,7740-BTPUX,Yes
755,2672-TGEFF,Yes


In [26]:
df_phone.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6361 entries, 0 to 6360
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   customerID     6361 non-null   object
 1   MultipleLines  6361 non-null   object
dtypes: object(2)
memory usage: 99.5+ KB


In [27]:
df_phone.describe()

,customerID,MultipleLines
count,6361,6361
unique,6361,2
top,0112-QAWRZ,No
freq,1,3390


# Limpieza de df_phone
 - Estandarizar los nombres de las columnas al formato `snake_case`.
 - Verificar si existen valores nulos o ausentes y definir la estrategia de imputación en caso necesario.
 - Revisar si hay registros duplicados.

In [28]:
#Conversión de nombres de columnas a estructura snake_case.
column_snake_case(df_phone)
print(df_phone.columns)

Index(['customer_id', 'multiple_lines'], dtype='object')


In [29]:
#Validación de datos nulos (NAN)
print(df_phone.isnull().sum())

customer_id       0
multiple_lines    0
dtype: int64


In [30]:
#Validación de duplicados
df_phone.duplicated().sum()

0

**El dataset quedó listo para unirse al resto de los datframes. Sin embargo, este dataset contiene menos registros (6,361) que algunos de los otros datasets revisados. Esto se debe a que no todos los clientes cuentan con servicios telefónicos, por lo que será necesario considerar este detalle al realizar la unión final de los datos.**

# Unión de los Datasets
 
 En este paso integraremos los cuatro datasets en un único DataFrame que consolide toda la información disponible sobre los clientes.
 
 Se utilizará la tabla de contratos como base principal del proceso de unión, ya que representa el núcleo de la relación entre el cliente y la empresa. De esta manera, se garantiza la inclusión de todos los registros de clientes, independientemente de si cuentan o no con servicios adicionales como Internet o telefonía.
 
 La clave de integración será la columna `customer_id`, presente en los cuatro datasets. Se aplicará una estrategia de unión tipo left join, que prioriza la conservación de todos los contratos activos y asocia la información complementaria disponible en las demás fuentes.
 
 Una vez realizada la fusión, se llevará a cabo una evaluación de valores ausentes para determinar su origen y establecer el tratamiento más adecuado, en función de su impacto en los análisis posteriores.

In [31]:
df_master = df_contract.merge(df_personal, on="customer_id", how="left")
df_master = df_master.merge(df_internet, on="customer_id", how="left")
df_master = df_master.merge(df_phone, on="customer_id", how="left")

In [32]:
df_master.sample(5)

,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,gender,senior_citizen,partner,dependents,internet_service,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,multiple_lines
4866,3688-FTHLT,2018-10-01,NaT,Month-to-month,Yes,Bank transfer (automatic),63.05,1067.05,Female,0,No,No,DSL,No,No,No,No,No,Yes,Yes
5933,6496-SLWHQ,2019-10-01,2020-01-01,Month-to-month,Yes,Electronic check,105.00,294.45,Male,1,No,No,Fiber optic,No,Yes,Yes,No,Yes,Yes,Yes
2228,3597-YASZG,2014-04-01,NaT,Two year,Yes,Bank transfer (automatic),104.45,7349.35,Female,1,Yes,No,Fiber optic,No,Yes,No,Yes,Yes,Yes,Yes
1452,0222-CNVPT,2015-10-01,NaT,Month-to-month,Yes,Credit card (automatic),48.80,2555.05,Male,1,No,No,DSL,Yes,No,No,No,Yes,Yes,NaN
5898,8277-RVRSV,2017-05-01,NaT,One year,No,Credit card (automatic),24.15,800.30,Female,0,Yes,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes


In [33]:
df_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7043 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   customer_id        7043 non-null   object        
 1   begin_date         7043 non-null   datetime64[ns]
 2   end_date           1869 non-null   datetime64[ns]
 3   type               7043 non-null   object        
 4   paperless_billing  7043 non-null   object        
 5   payment_method     7043 non-null   object        
 6   monthly_charges    7043 non-null   float64       
 7   total_charges      7043 non-null   float64       
 8   gender             7043 non-null   object        
 9   senior_citizen     7043 non-null   int64         
 10  partner            7043 non-null   object        
 11  dependents         7043 non-null   object        
 12  internet_service   5517 non-null   object        
 13  online_security    5517 non-null   object        
 14  online_b

# Transformación y corrección de datos.
 
 Después de unir los datasets, se identificaron valores nulos en las columnas relacionadas con los servicios de Internet y telefónicos. Esta situación refleja que ciertos clientes no cuentan con estos servicios contratados, y no representa una falla en la información sino una condición del negocio.
 
 La estrategia de imputación será la siguiente:
 
 - Para todas las columnas binarias relacionadas con los servicios de Internet y teléfono (por ejemplo, `online_security`, `online_backup`, `tech_support`, `multiple_lines`, etc.), se imputará el valor `"No"` indicando que el cliente no utiliza estos servicios.
 - Para la columna `internet_service`, que contiene el nombre del tipo de servicio contratado, se imputará el valor `"Sin servicio"` para diferenciar claramente los casos donde no hay servicio de Internet.
 
 De este modo, se estandarizarán los datos sin eliminar registros y hará que cada cliente esté representado de forma completa y precisa en el modelo de retención.

In [34]:
#Creo una lista de las columnas sin servicio.
no_service = [
    "online_security",
    "online_backup",
    "device_protection",
    "tech_support",
    "streaming_tv",
    "streaming_movies",
    "multiple_lines"
]

In [35]:
df_master[no_service] = df_master[no_service].fillna("No")

In [36]:
#Imputamos valores de la columna 'internet_service' con "Sin servicio"
df_master['internet_service'].fillna('Sin Servicio', inplace=True)

In [37]:
# checamos que se hayan imputado las columnas tratadas
df_master.isnull().sum()

customer_id             0
begin_date              0
end_date             5174
type                    0
paperless_billing       0
payment_method          0
monthly_charges         0
total_charges           0
gender                  0
senior_citizen          0
partner                 0
dependents              0
internet_service        0
online_security         0
online_backup           0
device_protection       0
tech_support            0
streaming_tv            0
streaming_movies        0
multiple_lines          0
dtype: int64

# Preprocesamiento de datos.
 
 Como primer paso del preprocesamiento, definiremos la variable objetivo que será utilizada para entrenar nuestro modelo de clasificación.
 
 La columna `end_date` nos indica si un cliente ha cancelado su contrato. Por lo tanto, a partir de esta columna crearemos una nueva variable binaria llamada `churn`, que tomará los siguientes valores:
 
 - `1` si el cliente ha cancelado (es decir, si `end_date` contiene una fecha).
 - `0` si el cliente sigue activo (es decir, si `end_date` es nulo o `NaT`).
 
 Esta nueva variable será nuestro objetivo el cual el modelo intentará predecir en función de las características del cliente y los servicios que utiliza.

In [38]:
df_master['churn'] = df_master['end_date'].notna().astype(int)

# **Ingeniería de Características: Duración del Contrato** 
 Dado que el AUC-ROC no superaba los 0.82 con distintos modelos, decidimos incorporar una nueva característica clave: `contract_duration` (duración del contrato en meses). Esta se calcula a partir de las fechas de inicio y fin del contrato. Para los clientes activos (aquellos sin `end_date`), se asume que su contrato continúa hasta la fecha más reciente de churn registrada en el dataset, más un día, o una fecha fija si no hay churns. Esta característica es vital ya que la duración del contrato puede ser un fuerte predictor de la propensión a la cancelación.

In [39]:
# Calcular contract_duration en meses
# Para clientes activos, se asume que el contrato continúa hasta la fecha más reciente de churn + 1 día,
# o una fecha predefinida si no hay clientes que hayan cancelado (para este dataset, hay churn).
max_end_date = df_master['end_date'].max()

if pd.isna(max_end_date):
    # Si no hay fechas de fin (nadie ha cancelado aún), usamos la fecha actual o una fecha de referencia posterior al último begin_date
    current_date = pd.to_datetime('2025-07-01') # Usamos una fecha posterior para cálculo si no hay churns
else:
    current_date = max_end_date + pd.Timedelta(days=1)

df_master['contract_duration'] = (
    (df_master['end_date'].fillna(current_date) - df_master['begin_date']).dt.days / 30.44
).round().astype(int)


print("Muestra de contract_duration:")
print(df_master[['begin_date', 'end_date', 'contract_duration', 'churn']].sample(5))

Muestra de contract_duration:
     begin_date   end_date  contract_duration  churn
2975 2019-11-01        NaT                  2      0
6817 2014-02-01        NaT                 71      0
3533 2019-04-01        NaT                  9      0
3241 2019-11-01 2019-12-01                  1      1
4116 2018-06-01        NaT                 19      0


# Análisis del equilibrio de clases

In [40]:
# ver la proporción de cada clase: cancelaron (1) y cuántos siguen activos (0)
df_master['churn'].value_counts(normalize=True)

0    0.73463
1    0.26537
Name: churn, dtype: float64

In [41]:
pd.crosstab(df_master['type'], df_master['churn'], normalize='index').round(3)

churn,0,1
type,,
Month-to-month,0.573,0.427
One year,0.887,0.113
Two year,0.972,0.028


# Transformar las variables categóricas en variables numéricas (one-hot encoding) para preparar los datos para modelado.

In [42]:
# Selección de columnas categóricas
cat_cols = df_master.select_dtypes(include='object').columns.drop(['customer_id'])  # No codificamos ID

# Aplicar one-hot encoding
df_model = pd.get_dummies(df_master, columns=cat_cols, drop_first=True)

In [43]:
# Validamos que el df tenga el mismo tamaño y que se haya aplicado el metodo Dummy
print(df_model.shape)
df_model.sample(5)

(7043, 26)


,customer_id,begin_date,end_date,monthly_charges,total_charges,senior_citizen,churn,contract_duration,type_One year,type_Two year,...,dependents_Yes,internet_service_Fiber optic,internet_service_Sin Servicio,online_security_Yes,online_backup_Yes,device_protection_Yes,tech_support_Yes,streaming_tv_Yes,streaming_movies_Yes,multiple_lines_Yes
2239,8069-RHUXK,2014-07-01,NaT,35.70,2545.70,0,0,66,0,1,...,1,0,0,1,0,0,1,0,0,0
2450,7602-MVRMB,2014-02-01,NaT,110.45,8058.85,0,0,71,0,1,...,1,1,0,1,0,1,1,1,1,1
4638,7817-BOQPW,2019-09-01,2019-11-01,75.55,166.30,0,1,2,0,0,...,0,1,0,0,0,0,0,0,0,1
1240,2120-SMPEX,2018-01-01,NaT,20.15,536.35,0,0,24,0,0,...,0,0,1,0,0,0,0,0,0,0
4296,4489-SNOJF,2017-01-01,2019-12-01,72.25,2568.55,0,1,35,0,0,...,1,0,0,1,0,1,1,1,0,1


# Escalado de variables numéricas

In [44]:
# Columnas numericas (no binarias). Ahora incluye 'contract_duration'.
num_cols = ['monthly_charges', 'total_charges', 'contract_duration']

# Escalado con StandardScaler: media = 0, desviación = 1
scaler = StandardScaler()
df_model[num_cols] = scaler.fit_transform(df_model[num_cols])

# Selección de variables para modelado
 
 - **`X`**: contiene todas las columnas predictoras, excepto:
   - `'churn'`: es la variable objetivo (target), no debe ir como entrada.
   - `'customer_id'`: es un identificador, no aporta valor predictivo.
   - `'end_date'`: se usó para construir `churn`, pero incluirla como predictor generaría fuga de información (*data leakage*).
   - `'begin_date'`: la duración del contrato ya la captura, y es una fecha.
 
 - **`y`**: es la variable objetivo, corresponde a la columna `'churn'`.


In [45]:
X = df_model.drop(['churn', 'customer_id', 'begin_date', 'end_date'], axis=1)
y = df_model['churn']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=12345, stratify=y
) #stratify=y mantiene el mismo equilibrio de clases

In [46]:
# Concatenamos X_train y y_train
train = pd.concat([X_train, y_train], axis=1)

# Separar clases
clase_mayoritaria = train[train.churn == 0]
clase_minoritaria = train[train.churn == 1]

# Re-muestrear la clase minoritaria para igualar la cantidad
clase_minoritaria_upsampled = resample(clase_minoritaria,
                                       replace=True,
                                       n_samples=len(clase_mayoritaria),
                                       random_state=12345)

# Unir clases
train_balanceado = pd.concat([clase_mayoritaria, clase_minoritaria_upsampled])

# Nuevos X e y
X_train_resampled = train_balanceado.drop('churn', axis=1)
y_train_resampled = train_balanceado['churn']

# Modelos
 Entrenaremos el primer modelo base, el cual será Regresión Logistica.
 
 - Sirve como línea base para comparar con modelos más complejos después.
 - Es sensible al desbalance, por lo que usaremos `class_weight='balanced'` para compensar eso desde el inicio.

In [47]:
# Entrenamos el modelo
model = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=12345)
model.fit(X_train_resampled, y_train_resampled)

# Predicciones
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]  # Para métricas probabilísticas

# Evaluación
print(" Classification Report (Logistic Regression - Baseline):")
print(classification_report(y_test, y_pred))

print(" Confusion Matrix (Logistic Regression - Baseline):")
print(confusion_matrix(y_test, y_pred))

print(" ROC AUC Score (Logistic Regression - Baseline):")
print(round(roc_auc_score(y_test, y_proba), 3))

 Classification Report (Logistic Regression - Baseline):
              precision    recall  f1-score   support

           0       0.91      0.71      0.80      1035
           1       0.50      0.80      0.62       374

    accuracy                           0.74      1409
   macro avg       0.71      0.76      0.71      1409
weighted avg       0.80      0.74      0.75      1409

 Confusion Matrix (Logistic Regression - Baseline):
[[739 296]
 [ 75 299]]
 ROC AUC Score (Logistic Regression - Baseline):
0.83


In [48]:
# Para este script, ya se escalaron df_model antes de X/y split y luego se resampleó.
# Dado que 'df_model' ya fue escalado, X_train_resampled y X_test ya contienen los datos escalados.

# Modelos

#Random Forest
rf = RandomForestClassifier(random_state=12345, class_weight='balanced') 

# XGBoost
xgb = XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=12345,
    scale_pos_weight=len(y_train_resampled[y_train_resampled==0])/len(y_train_resampled[y_train_resampled==1]) 
)

# Catboost
cat = CatBoostClassifier(verbose=0, random_state=12345, auto_class_weights='Balanced') 

# LightGBM
lgbm = LGBMClassifier(random_state=12345, scale_pos_weight=len(y_train_resampled[y_train_resampled==0])/len(y_train_resampled[y_train_resampled==1])) 

# Función de evaluación
def evaluar_modelo(nombre, modelo, X_train, y_train, X_test, y_test):
    print(f"\n-------------------- {nombre} -----------------")
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)
    y_proba = modelo.predict_proba(X_test)[:, 1]

    print("Accuracy:", round(modelo.score(X_test, y_test), 4))
    print("Matriz de confusión:")
    print(confusion_matrix(y_test, y_pred))
    print("Reporte de clasificación:")

    #classification_report:
    # proporciona un resumen de las principales métricas (precisión, recall, f1-score) para cada clase, y luego las agrega utilizando promedios.
    print(classification_report(y_test, y_pred))
    print("ROC AUC:", round(roc_auc_score(y_test, y_proba), 4))

# Evaluación de todos los modelos (con balanceo de clases)
evaluar_modelo("Random Forest", rf, X_train_resampled, y_train_resampled, X_test, y_test)
evaluar_modelo("XGBoost", xgb, X_train_resampled, y_train_resampled, X_test, y_test)
evaluar_modelo("CatBoost", cat, X_train_resampled, y_train_resampled, X_test, y_test)
evaluar_modelo("LightGBM", lgbm, X_train_resampled, y_train_resampled, X_test, y_test)


-------------------- Random Forest -----------------
Accuracy: 0.8133
Matriz de confusión:
[[910 125]
 [138 236]]
Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.87      0.88      0.87      1035
           1       0.65      0.63      0.64       374

    accuracy                           0.81      1409
   macro avg       0.76      0.76      0.76      1409
weighted avg       0.81      0.81      0.81      1409

ROC AUC: 0.8592

-------------------- XGBoost -----------------
Accuracy: 0.8297
Matriz de confusión:
[[893 142]
 [ 98 276]]
Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.90      0.86      0.88      1035
           1       0.66      0.74      0.70       374

    accuracy                           0.83      1409
   macro avg       0.78      0.80      0.79      1409
weighted avg       0.84      0.83      0.83      1409

ROC AUC: 0.8844

-------------------- CatBoost -------

# Conclusión Previa

Tras evaluar todos los modelos entrenados, se observó que CatBoost alcanzó el mejor desempeño general, con una métrica ROC AUC de 0.8915, superando el umbral requerido de 0.88. Además, logró un buen balance entre precisión (0.63), recall (0.78) y f1-score (0.70) para la clase positiva (clientes que cancelan), lo cual es clave para el objetivo del negocio.

La matriz de confusión indica que el modelo clasificó correctamente a 866 de 1035 clientes que no cancelaron (clase 0), lo que representa una tasa de acierto del 83.7% en esta clase. En cuanto a los clientes que sí cancelaron (clase 1), identificó correctamente a 292 de 374, logrando una tasa de recall del 78.1%. Esto sugiere que el modelo tiene un buen desempeño tanto en detectar la permanencia como la cancelación, siendo especialmente útil para identificar clientes en riesgo de churn con un balance aceptable entre precisión y sensibilidad.

Por tanto, CatBoost se selecciona como el modelo final, ya que ofrece una excelente capacidad predictiva para anticipar la cancelación de clientes.